## A. Load the business.json file
Set up the business.json environment path to load using Pandas

In [1]:
import pandas as pd

business_json_path = 'business.json'
business = pd.read_json(business_json_path, lines=True)

In [2]:
print(business.shape)
business.head()

(192609, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


### Clean the business.json file
Only keep the businesses that are still open in the dataset

In [17]:
# Make sure the Pandas version is above 0.25
# If not, upgrade Pandas version
# !pip3 install --upgrade pandas
pd.__version__

'0.25.3'

In [4]:
# There are significant amount of businesses that are not open anymore
business.is_open.value_counts()

1    158525
0     34084
Name: is_open, dtype: int64

In [6]:
# Keep only business that are still open
# 1 = open, 0 = closed
# Drop columns that may not be relavent
df_business = business[business['is_open']==1].drop(['hours','is_open','review_count','latitude','longitude','attributes'], axis=1)
print(df_business.shape)
df_business.head()

(158525, 8)


,business_id,name,address,city,state,postal_code,stars,categories
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,2.5,"Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,4.0,"Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,5.0,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,4.0,"Plumbing, Shopping, Local Services, Home Servi..."
5,68dUKd8_8liJ7in4aWOSEA,The UPS Store,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd",Mississauga,ON,L5V 0B1,2.5,"Shipping Centers, Couriers & Delivery Services..."


### Split the businesses by category
We can find the relevant reviews that we want later on by querying for the relevant businesses first using categories.
For this project, we are only interested in restaurants related businesses.

In [7]:
# df.explode requires pandas ver 0.25
# Create one row for each series that contain comma-separated items
df_explode = df_business.assign(categories = df_business.categories.str.split(', ')).explode('categories')
df_explode.sample(3)

,business_id,name,address,city,state,postal_code,stars,categories
138506,1VM4RVa2l6OYFhtUM9jwFw,Bacchus Wine And Tapas,"138 Village View Dr, Ste 107",Mooresville,NC,28117,3.5,Restaurants
90240,rtCXyvG6iy47sdn7Ku6DVQ,Patrons Barber Shop,2029 W Glendale Ave,Phoenix,AZ,85021,4.5,Beauty & Spas
170930,my7WOW-Dq7eV4OxfxmlI-g,Burger King,11002 Lorain Ave,Cleveland,OH,44111,3.0,Fast Food


In [11]:
print('Total number of categories:', len(df_explode.categories.value_counts()))
print('Top 20 categories:')
df_explode.categories.value_counts()[:20]

Total number of categories: 1290
Top 10 categories:


Restaurants                  42237
Shopping                     26734
Food                         23208
Home Services                18455
Beauty & Spas                16545
Health & Medical             15875
Local Services               12830
Automotive                   11955
Nightlife                     9396
Event Planning & Services     8960
Active Life                   8217
Bars                          8155
Fast Food                     6409
Fashion                       6293
Hair Salons                   5919
Professional Services         5844
Sandwiches                    5753
Coffee & Tea                  5731
Home & Garden                 5667
Auto Repair                   5562
Name: categories, dtype: int64

In [15]:
# Keep only business with categories that are food related
business_res = df_business[df_business['categories'].str.contains(
              'Restaurants|Food',
              case=False, na=False)]

In [16]:
print(business_res.shape)
business_res.head()

(54459, 8)


,business_id,name,address,city,state,postal_code,stars,categories
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,2.5,"Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,4.0,"Sushi Bars, Restaurants, Japanese"
11,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,3.0,"Restaurants, Breakfast & Brunch, Mexican, Taco..."
13,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,5981 Andrews Rd,Mentor-on-the-Lake,OH,44060,4.0,"Italian, Restaurants, Pizza, Chicken Wings"
14,-K4gAv8_vjx8-2BxkVeRkA,Baby Cakes,4145 Erie St,Willoughby,OH,44094,3.0,"Bakeries, Food"


## B. Loading and cleaning the review.json file
Set up the review.json environment path to load using Pandas.

In [18]:
# Set up your local path
review_json_path = 'review.json'

In [19]:
# Set chunk size (smaller if dataset is smaller)
# 2019 Yelp review.json has more than 6 million reviews(rows)
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      # identifying the data type of each column can reduce memory usage
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

## C. Merge Review.json and Business.json file
By merging only the relevant businesses to the review file, the final dataset will only consist of reviews from those businesses.

In [20]:
# There are multiple chunks to be read
chunk_list = []
for chunk in review:
    # Drop columns that aren't needed
    chunk = chunk.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk = chunk.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_res, chunk, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
    


589309 out of 1,000,000 related reviews
580408 out of 1,000,000 related reviews
588231 out of 1,000,000 related reviews
561564 out of 1,000,000 related reviews
593434 out of 1,000,000 related reviews
585462 out of 1,000,000 related reviews
396057 out of 1,000,000 related reviews


In [21]:
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
print(df.shape)
df.sample(3)

(3894465, 12)


,business_id,name,address,city,state,postal_code,stars,categories,user_id,review_stars,text,date
2119962,nUpz0YiBsOK7ff9k3vUJ3A,Buddy V's Ristorante,3327 S Las Vegas Blvd,Las Vegas,NV,89109,4.0,"American (Traditional), Restaurants, Italian",EsOu51dW3UTDJTApBxwz2g,5,Discovered Lunch Express Menu\n\nI had done so...,2016-02-24 16:07:07
3033859,ecn5dq7bFA6wCeeNnXxSzw,Pei Wei,"742 E Glendale Ave, Ste 110",Phoenix,AZ,85020,2.5,"Asian Fusion, Chinese, Fast Food, Restaurants,...",qwsTVLtAFBDl7MORXpe9Vw,4,"I love how fast this location is, hardly crowd...",2015-01-30 22:06:31
2631136,dqeUvrlMxTAk0VL2n1ukaw,New Asian Star,"2755 S 99th Ave, Ste 102",Tolleson,AZ,85353,4.0,"Restaurants, Chinese, Food, Nightlife",9cx7T4nHHUS5ToAP8HMfMA,5,Best Chinese food in all of Arizona!!! Mongoli...,2015-01-15 04:58:44


## D. Convert the new data frame into a CSV file

In [22]:
#csv_name = "yelp_restaurants.csv"
#df.to_csv(csv_name, index=False)